In [77]:
%reload_ext autoreload
%autoreload 2
from modules.evaluation import generate_examples, llm_assisted_evaluation
from modules.document_transformers import add_metadata_to_documents
from modules.loaders import get_reddit_loader, get_wiki_loader, get_custom_flipper_loader
from modules.retrievers import get_tavily_retriever, vector_store_retriever_from_loaders
from modules.evaluation import generate_examples, llm_assisted_evaluation
from modules.vector_stores import chroma_vector_store, FAISS_vector_store, doc_array_in_memory_store
from modules.utils import save_docs_to_jsonl, load_docs_from_jsonl
from langchain_openai import ChatOpenAI

In [3]:

import os

tavily_api_key = os.environ.get('TAVILY_API_KEY', '')
api_key = os.environ.get('OPENAI_API_KEY', '')

if not api_key:
    print('You need to set the OPENAI_API_KEY environment variable to use this script.')
if not tavily_api_key:
    print('You need to set the TAVILY_API_KEY environment variable to use this script.')

In [4]:
llm_model = "gpt-4-0125-preview"

# Loading the data

In [16]:
flipper_loader = get_custom_flipper_loader()
reddit_loader = get_reddit_loader()

In [48]:
#import nest_asyncio
#nest_asyncio.apply()

flipper_loader.requests_per_second = 2
flipper_docs = flipper_loader.load()

In [55]:

flipper_docs[1]

Document(page_content='\n\nPinball Archive Rule Sheet: The Addams Family\n\n\n\n\n\n\n\n\xa0 The Addams Family\n\n\n\n\n\n\n\n\n\nVersion 2.0.\nLast revised January 27, 1995.\nCompiled and written by Brian Dominy.\n\n\nIntroduction\n\nThis is version 2.0 of my Guide to\nThe Addams Family pinball game. No major\nchanges have been made from the previous version, although\nthere is more detailed information about a few things, including\npreliminary information about TAF Gold,\nthe 1994 collector\'s edition of the game that has\nbeen produced.\n Comments and suggestions are always welcome. You can send\ne-mail to gt6139a@prism.gatech.edu\nCredit to those people who have contributed in some way to\nthis guide appears at the end.\n These rules describe play on normal default settings and\non machines without broken diverters or faulty switches. See\nthe section on "Defaults" for a listing of defaults and operator-settable\noptions. See your operator if anything\'s broken :-)\n This is a spe

# Add metadata

In [60]:
enhanced_flipper_docs = add_metadata_to_documents(flipper_docs)
enhanced_flipper_docs[:5]

[Document(page_content='\n\nPinball Archive Rule Sheet: 4 Square\n\n\n\n\n\n\n\n\xa0 4 Square\n\n\n\n\n\n\n\n\n\nWritten by Tim Meighan\n\n\n"4 Square" was released by D. Gottlieb & Co. in 1971.\nMore information is available from the\nInternet Pinball Database.\n\nGAME ELEMENTS\n\nThe 4 Square playfield includes two sets of targets and three sets\nof rollovers, with each set consisting of four items.  Each item\nin a given set has a corresponding indicator light marked with a\nunique number value: 1, 2, 3, or  4.  The upper central area of the\nplayfield also contains three pop bumpers arranged in a triangle\nconfiguration.  There are also two kicking rubber islands located\nalong each side just above the flippers.  The lower central portion\nof the playfield is wide open and contains four rows of sequence\nindicator lights, with each row consisting of 4 lights numbered\nfrom 1 to 4.\n\nSCORING\n\nAll rollovers and targets score 50 points.  During the game, certain\nsets of targets an

In [63]:
len(enhanced_flipper_docs)

188

In [62]:
save_docs_to_jsonl(enhanced_flipper_docs, 'data/flipper_docs_with_metadata.jsonl')

In [64]:
enhanced_flipper_docs2 = load_docs_from_jsonl('data/flipper_docs_with_metadata.jsonl')
len(enhanced_flipper_docs2)

188

In [65]:
import json
print(
    *[json.dumps(d.metadata) for d in enhanced_flipper_docs[:5]],
    sep="\n\n---------------\n\n",
)

{"pinball_title": "4 Square", "manufacturer": "D. Gottlieb & Co.", "source": "http://www.pinball.org/rules/4square.html", "title": "Pinball Archive Rule Sheet: 4 Square", "language": "No language found."}

---------------

{"pinball_title": "The Addams Family", "manufacturer": "Williams/Bally, Inc.", "source": "http://www.pinball.org/rules/addamsfamily.html", "title": "Pinball Archive Rule Sheet: The Addams Family", "language": "No language found."}

---------------

{"pinball_title": "The Addams Family Gold Edition", "manufacturer": "Williams", "source": "http://www.pinball.org/rules/addamsfamilygold.html", "title": "Pinball Archive Rule Sheet: The Addams Family Gold Edition", "language": "No language found."}

---------------

{"pinball_title": "Apollo 13", "possible_shots": "D-O-C-K standup targets, Left orbit, Food standup, Spinner lane, Rocket ramp, Battery standup targets, M-O-O-N standups, Blastoff ramp, Water standup, Right orbit, Mission start lane", "manufacturer": "Sega", "s

In [69]:
#chroma_db = chroma_vector_store(flipper_docs, api_key)
faiss_db = FAISS_vector_store(enhanced_flipper_docs, api_key)

Created a chunk of size 1161, which is longer than the specified 1000
Created a chunk of size 1127, which is longer than the specified 1000
Created a chunk of size 1837, which is longer than the specified 1000
Created a chunk of size 1119, which is longer than the specified 1000
Created a chunk of size 1194, which is longer than the specified 1000
Created a chunk of size 3619, which is longer than the specified 1000
Created a chunk of size 1498, which is longer than the specified 1000
Created a chunk of size 1172, which is longer than the specified 1000
Created a chunk of size 1238, which is longer than the specified 1000
Created a chunk of size 2753, which is longer than the specified 1000
Created a chunk of size 3297, which is longer than the specified 1000
Created a chunk of size 2123, which is longer than the specified 1000
Created a chunk of size 1327, which is longer than the specified 1000
Created a chunk of size 1207, which is longer than the specified 1000
Created a chunk of s

In [71]:
faiss_db.similarity_search("All possible shots in Medival Madness", 5)

[Document(page_content='Look for Barnyard Madness.  Think of something difficult that the game is\nchallenging you to do... complete it and you will have access to Barnyard\nMadness.  Chickens!  Horses!  Pigs!  Oh my!\n\nAdditionally, it is possible to change the critters being saved in Video\nMode.  Throw cows!  Lots of cows!\n\nEntering Initials\nBesides the regular high score board (2 credits for Grand Champion, 1 credit\nfor High Scores 1-4), Medieval Madness allows you to enter your initials for:\n\n\n Master of Madness: highest total score in one Multiball Madness\n Castle Crusher: most Castles destroyed\n Joust Champion: most Joust Victories\n Defender of Damsels: most Damsels saved\n Catapult Ace: most Catapult Slams\n Patron of the Peasants: most Peasant Revolts\n Troll Avenger: highest Troll total in game', metadata={'pinball_title': 'Medieval Madness', 'manufacturer': 'Williams', 'source': 'http://www.pinball.org/rules/medievalmadness.html', 'title': 'Pinball Archive Rule Sh

# Create retriever tools for the Agent

In [74]:
flipper_retriever = faiss_db.as_retriever()

In [137]:
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults


retriever_tool = create_retriever_tool(
    flipper_retriever,
    "pinball_search",
    "Search for information about Pinball! Use metadata if it helps to answer the question. Do not combine multiple sources of information to produce answers.",
)

tavity_tool = TavilySearchResults(name="tavily_search")


# Create the Agents

In [139]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

In [140]:
# Baseline agent
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [tavity_tool]

prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent_baseline = create_openai_functions_agent(llm, tools, prompt)
agent_executor_baseline = AgentExecutor(agent=agent_baseline, tools=tools, verbose=True)

In [173]:
# Custom agent
# Get the prompt to use - you can modify this!
tools = [retriever_tool]
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a pinball agent, always giving as short and concise answers as possible to pinball-related questions. You do not know anything else but pinball. You can use the `pinball_search` tool to find information about pinball.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
llm = ChatOpenAI(model=llm_model, temperature=0)
agent_custom = create_openai_functions_agent(llm, tools, prompt)


In [169]:
# Combined agent
tools = [retriever_tool, tavity_tool]
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a pinball agent, always giving as short and concise answers as possible to pinball-related questions. You have two ways of finding information. `pinball_search` is great for finding rules and general information about the game. The `tavity_search` tool can be used to find strategy and gameplay related information, or more information if pinball search doesn't return anything useful.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
llm = ChatOpenAI(model=llm_model, temperature=0)
agent_combined = create_openai_functions_agent(llm, tools, prompt)

In [182]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
tools = [retriever_tool]
llm_with_tools = llm.bind_tools(tools)


from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

#agent = create_openai_functions_agent(llm, tools, prompt)
agent_chained = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [144]:
agent_executor_baseline = AgentExecutor(agent=agent_baseline, tools=[tavity_tool], verbose=True)

In [174]:
agent_executor_custom = AgentExecutor(agent=agent_custom, tools=[retriever_tool], verbose=True)

In [146]:
agent_executor_combined = AgentExecutor(agent=agent_combined, tools=[retriever_tool,tavity_tool], verbose=True)

In [183]:
agent_executor_chained = AgentExecutor(agent=agent_chained,  tools=[retriever_tool], verbose=True)

# Evaluate the different Agents

In [176]:
import langchain
langchain.debug = False
agent_executor_baseline.invoke({"input": "How do you get a multi-ball in Addams Family?"})



> Entering new AgentExecutor chain...
In the pinball game "The Addams Family," getting a multi-ball is a key objective for players to score more points and increase their chances of achieving a high score. Here's how you can get a multi-ball in "The Addams Family" pinball game:

1. **Light the Mansion Rooms**: To start a multi-ball, you need to light all the Mansion Rooms by hitting the corresponding shots on the playfield. Each shot corresponds to a different room in the mansion.

2. **Shoot the Vault**: Once you have lit all the Mansion Rooms, shoot the ball into the Vault. This will start the multi-ball mode.

3. **Keep the Balls in Play**: During multi-ball, you will have multiple balls in play at the same time. Your goal is to keep as many balls in play as possible and score points by hitting various targets on the playfield.

4. **Maximize Your Score**: Use the multi-ball mode to score as many points as you can by hitting jackpot shots and other high-scoring targets on the play

{'input': 'How do you get a multi-ball in Addams Family?',
 'output': 'In the pinball game "The Addams Family," getting a multi-ball is a key objective for players to score more points and increase their chances of achieving a high score. Here\'s how you can get a multi-ball in "The Addams Family" pinball game:\n\n1. **Light the Mansion Rooms**: To start a multi-ball, you need to light all the Mansion Rooms by hitting the corresponding shots on the playfield. Each shot corresponds to a different room in the mansion.\n\n2. **Shoot the Vault**: Once you have lit all the Mansion Rooms, shoot the ball into the Vault. This will start the multi-ball mode.\n\n3. **Keep the Balls in Play**: During multi-ball, you will have multiple balls in play at the same time. Your goal is to keep as many balls in play as possible and score points by hitting various targets on the playfield.\n\n4. **Maximize Your Score**: Use the multi-ball mode to score as many points as you can by hitting jackpot shots an

In [177]:
agent_executor_custom.invoke(
    {
        "input": "How do you get a multi-ball in Addams Family?",
    }
)


Invoking: `pinball_search` with `{'query': 'Addams Family pinball multi-ball'}`


This is version 2.0 of my Guide to
The Addams Family pinball game. No major
changes have been made from the previous version, although
there is more detailed information about a few things, including
preliminary information about TAF Gold,
the 1994 collector's edition of the game that has
been produced.
 Comments and suggestions are always welcome. You can send
e-mail to gt6139a@prism.gatech.edu
Credit to those people who have contributed in some way to
this guide appears at the end.
 These rules describe play on normal default settings and
on machines without broken diverters or faulty switches. See
the section on "Defaults" for a listing of defaults and operator-settable
options. See your operator if anything's broken :-)
 This is a special version of this rulesheet formatted especially
for the World Wide Web. Your comments on this new presentation
of the rules are also welcome.
 The Addams Family is a

{'input': 'How do you get a multi-ball in Addams Family?',
 'output': "To start Multiball in The Addams Family pinball, you need to lock balls which can be challenging to do intentionally. The game's design makes it difficult to aim directly for locks, often resulting in balls accidentally locking while aiming for other targets. Once you manage to lock the required balls, Multiball begins with a notable start quote and a ball served to the plunger for a potential skill shot. The locked balls are then released, and the objective shifts to hitting either the Train for a Jackpot or the left ramp for a Super Jackpot. The Super Jackpot value increases with each subsequent Multiball, starting as a double Jackpot and increasing to a triple Jackpot for each additional Multiball."}

In [179]:
agent_executor_combined.invoke({"input": "What is the best strategy in Addams Family?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': 'Addams Family pinball best strategy'}`


[{'url': 'https://www.wired.com/story/addams-family-pinball/', 'content': "Manufactured by Midway in the early 90s and inspired by the film of that era, it's the best selling pinball machine of all time. According to Wikipedia , a total of 20,270 units have been sold."}, {'url': 'https://www.reddit.com/r/pinball/comments/dkth5c/advanced_the_addams_family_pinball_strategies/', 'content': "Halloween themed pinball tournament coming up that includes The Addams Family Special Collector's Edition. I know the original game very well, but seeking tips or strategies unique to the Collector's Edition or pinball competition in general. The format allows you to decide when you want to start your round, then you get 3 back-to-back games."}, {'url': 'https://www.youtube.com/watch?v=tuJifflfuuA', 'content': "Gameplay video of 'The Addams Family' by Bally, 1992.The Addams Famil

{'input': 'What is the best strategy in Addams Family?',
 'output': 'The best strategy in Addams Family pinball focuses on getting extra balls to extend your game. Here are ways to light an extra ball at the Thing ramp:\n- X Bear ramp: X defaults to 8 but can vary between 6 and 12.\n- 50 Bear ramps.\n- Light extra ball mansion room.\n- Tour the Mansion: This must be collected.\n\nFocusing on these objectives can help you achieve a longer game and higher scores.'}

In [186]:
agent_executor_chained.invoke({"input": "How do you get a multi ball in Addams Family?"})



> Entering new AgentExecutor chain...

Invoking: `pinball_search` with `{'query': 'Addams Family pinball multi ball'}`


This is version 2.0 of my Guide to
The Addams Family pinball game. No major
changes have been made from the previous version, although
there is more detailed information about a few things, including
preliminary information about TAF Gold,
the 1994 collector's edition of the game that has
been produced.
 Comments and suggestions are always welcome. You can send
e-mail to gt6139a@prism.gatech.edu
Credit to those people who have contributed in some way to
this guide appears at the end.
 These rules describe play on normal default settings and
on machines without broken diverters or faulty switches. See
the section on "Defaults" for a listing of defaults and operator-settable
options. See your operator if anything's broken :-)
 This is a special version of this rulesheet formatted especially
for the World Wide Web. Your comments on this new presentation
of the rules a

{'input': 'How do you get a multi ball in Addams Family?',
 'output': "The search did not provide specific details on how to achieve multi-ball in The Addams Family pinball game. Typically, multi-ball modes in pinball games are activated by completing certain objectives, such as hitting specific targets a number of times or completing a sequence of shots. For detailed instructions on activating multi-ball in The Addams Family, consulting the game's manual or a dedicated pinball guide for this specific machine would be recommended."}

In [192]:

example_genertor_llm = ChatOpenAI(model=llm_model, temperature=0)
new_samples = generate_examples(example_genertor_llm, enhanced_flipper_docs)

/Users/tobiaslofgren/Desktop/code/venvs/flipper_ama/lib/python3.11/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


New examples: [{'qa_pairs': {'query': 'In the game of "4 Square" by D. Gottlieb & Co., released in 1971, how does the scoring system for the pop bumpers work, and how does their lighting condition affect their score value?', 'answer': 'In "4 Square," the left and right pop bumpers score 1 point when unlit and 10 points when lit. The central pop bumper scores 10 points when unlit and 100 points when lit. The three bumpers never all light up at the same time. They alternate between having both 10 point bumpers lit and the 100 point bumper out, or the 100 point bumper lit and both 10 point bumpers out. This alternation occurs when 1 point is scored, either by hitting a kicking rubber or when either 10 point bumper is hit while not lit. Once the 10 point bumpers are lit, all three bumpers score 10 points, and none of the bumpers can cause the lights to alternate again. In this situation, a player must hit a kicking rubber to switch the bumper lights back, lighting up the 100 point bumper a

In [196]:
with open('data/pinball_examples.json', 'w') as file:
    json.dump(new_samples, file)

In [198]:
hard_coded_samples = json.load(open('data/manual_pinball_examples.json'))

In [200]:
total_samples = new_samples + hard_coded_samples

In [202]:
predictions = agent_executor_chained.apply(total_samples)



> Entering new AgentExecutor chain...


/Users/tobiaslofgren/Desktop/code/venvs/flipper_ama/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `apply` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use batch instead.
  warn_deprecated(


KeyError: 'input'